In [1]:
import pandas as pd

df = pd.read_csv("../data/train.csv")
df.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False


In [2]:
#PRINTING NUMBER OF FEATURES AND NUMBER OF ROWS
rows, cols = df.shape
print("Rows:", rows)
print("Features:", cols)

Rows: 101763
Features: 23


In [4]:
#CHECKING FOR NULL VALUES
df.isnull().sum()

id                   0
loc                  0
v(g)                 0
ev(g)                0
iv(g)                0
n                    0
v                    0
l                    0
d                    0
i                    0
e                    0
b                    0
t                    0
lOCode               0
lOComment            0
lOBlank              0
locCodeAndComment    0
uniq_Op              0
uniq_Opnd            0
total_Op             0
total_Opnd           0
branchCount          0
defects              0
dtype: int64

In [6]:
import numpy as np
import pandas as pd

summary = []

for col in df.columns:
    series = df[col]
    
    # Determine type
    if series.dtype == 'object':
        col_type = "categorical"
    elif series.dtype == 'bool':
        col_type = "boolean"
    else:
        col_type = "numerical"
    
    # Values or range
    if col_type == "categorical":
        values = ", ".join(map(str, series.unique()[:10]))
    elif col_type == "boolean":
        values = str(series.unique().tolist())
    else:
        values = f"{series.min()} – {series.max()}"
    
    # Missing count
    missing = series.isnull().sum()
    
    # Outlier detection
    if col_type == "numerical":
        try:
            q1 = series.quantile(0.25)
            q3 = series.quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            outliers = ((series < lower) | (series > upper)).sum()
        except Exception:
            outliers = "error"
    else:
        outliers = "N/A"
    
    summary.append([col, col_type, values, missing, outliers])

summary_df = pd.DataFrame(summary, columns=["Feature", "Type", "Values / Range", "Missing", "Outliers"])
summary_df


,Feature,Type,Values / Range,Missing,Outliers
0,id,numerical,0 – 101762,0,0
1,loc,numerical,1.0 – 3442.0,0,8954
2,v(g),numerical,1.0 – 404.0,0,9029
3,ev(g),numerical,1.0 – 165.0,0,10855
4,iv(g),numerical,1.0 – 402.0,0,6954
5,n,numerical,0.0 – 8441.0,0,7834
6,v,numerical,0.0 – 80843.08,0,8883
7,l,numerical,0.0 – 1.0,0,5192
8,d,numerical,0.0 – 418.2,0,5490
9,i,numerical,0.0 – 569.78,0,4885


In [8]:
df["defects"].unique()

array([False,  True])

In [10]:
df["defects"] = df["defects"].astype(int)

In [11]:
df["defects"].unique()

array([0, 1])

In [12]:
print(df["defects"].value_counts())
print(df["defects"].value_counts(normalize=True))

defects
0    78699
1    23064
Name: count, dtype: int64
defects
0    0.773356
1    0.226644
Name: proportion, dtype: float64


In [14]:
df.to_csv("../data/train_clean.csv", index=False)